<a href="https://colab.research.google.com/github/Mizuto07/Prog2kakushin/blob/main/work1_screiping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pandas  
work1で使用するfoods.xlsxファイルを作成するスクレイピングプログラムです。  
クックパッドから料理に関する情報を抽出し、エクセルに順に書き込んでいきます。  
試していただくために処理を短くしているので、おそらく5-10分で実行完了すると思います。  
また、クックパッドの利用規約によると、禁止事項として「通常に本サービスを利用する行為を超えてサーバーに負荷をかける行為もしくはそれを助長するような行為、その他本サービスの運営・提供もしくは他の利用者による本サービスの利用を妨害し、またはそれらに支障をきたす行為」と書かれているのでスクレイピングをする際は1つのデータを取るときにtime.sleep()で20秒間隔をあけてサーバーに負荷をかけないようにします。そしてrobots.txtによると、料理検索結果ページの/serchに対するスクレイピングは禁止されていません。

In [1]:
!pip install pyqrcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 423 kB/s 
  Created wheel for pyqrcode: filename=PyQRCode-1.2.1-py3-none-any.whl size=36245 sha256=b6172e6cd720d62f012d4d1d04dca19519ba061efcc2b9ae42313ee5f8ffe8e0
  Stored in directory: /root/.cache/pip/wheels/e6/5e/b1/e42b38f43c94593ea57059de7e585c009c0f54e36e1a532992
Successfully built pyqrcode


In [2]:
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import re
import datetime
import pyqrcode as qr

In [ ]:
waitTime = 20 # 待機時間
#pageNum = 5 # ひとつのURLに対し何ページ読むか
pageNum = 1

wb = openpyxl.Workbook()
wb.save('foods.xlsx')

# エクセル書き込みの時に使う行番号
R = 1

url_list = []

images = []

# 、で分割した具材を入れるリスト
n_ings = []


# エクセルファイルに書き込むときの禁止文字を削除する関数 https://qiita.com/que9/items/8326133721500741bde7
def illegal_char_remover(data):

    ILLEGAL_CHARACTERS_RE = re.compile(

        r'[\000-\010]|[\013-\014]|[\016-\037]|[\x00-\x1f\x7f-\x9f]|[\uffff]')

    '''Remove ILLEGAL CHARACTER.'''

    if isinstance(data, str):
        return ILLEGAL_CHARACTERS_RE.sub("", data)
    else:
        return data

# ファイル名に使えない記号リスト
forbiddenSymbls = ['\\', '/', ':', '*', '?', '"', '<', '>', '|', '.']


# URL作成
URL1 = 'https://cookpad.com/search/%E7%89%9B%E8%82%89?order=&recipe_hit=90278' # 牛肉の検索ページ
#URL2 = 'https://cookpad.com/search/%E8%B1%9A%E8%82%89?order=&recipe_hit=313467' # 豚肉
#URL3 = 'https://cookpad.com/search/%E9%B6%8F%E8%82%89?order=&recipe_hit=394128' # 鶏肉
#URL4 = 'https://cookpad.com/search/%E3%82%B8%E3%83%A3%E3%82%AC%E3%82%A4%E3%83%A2?order=&recipe_hit=211675' # じゃがいも
#URL5 = 'https://cookpad.com/search/%E4%BA%BA%E5%8F%82?order=&recipe_hit=425380' # にんじん
#URL6 = 'https://cookpad.com/search/%E7%8E%89%E3%81%AD%E3%81%8E?order=&recipe_hit=602561' # 玉ねぎ
#URL7 = 'https://cookpad.com/search/%E3%82%AD%E3%83%A3%E3%83%99%E3%83%84?order=&recipe_hit=186069' # キャベツ
#URL8 = 'https://cookpad.com/search/%E3%82%B7%E3%83%BC%E3%83%81%E3%82%AD%E3%83%B3?order=&recipe_hit=98122' # シーチキン
#tempList = [URL1, URL2, URL3, URL4, URL5, URL6, URL7, URL8]
tempList = [URL1]


for i in range(len(tempList)):
    num = tempList[i].find('?')
    num2 = tempList[i].find('recipe_hit')
    for j in range(pageNum):
        if j == 0:
            url_list.append(tempList[i])
        else:
            url_list.append(f'{tempList[i][:num+1]}order=&page={j+1}&{tempList[i][num2:]}')
# print(url_list)


# スクレイピング本処理
# url_listの中に入ったすべてのURL分繰り返す
for s in range(len(url_list)):

    titles = []
    foodURLs = []
    ingredients = []
    r = requests.get(url_list[s])
    soup = BeautifulSoup(r.content, 'html.parser')  

    foodTitles = []
    foodIngredients = []
    # foodImage = []

    # htmlから情報を取得
    foodTitles = soup.find_all(class_="recipe-title")
    time.sleep(waitTime)
    foodIngredients = soup.find_all(class_="ingredients")
    #time.sleep(waitTime)
    #foodImage = soup.find_all(class_ = 'recipe-image')  
    

    for title in foodTitles: # タイトルと料理のページURLをリストに保管  
        
        ti = title.get_text()


        ti = illegal_char_remover(ti)
        for p in range(len(forbiddenSymbls)): # 料理タイトルからファイル名にできない記号を除去

            if forbiddenSymbls[p] in ti:
                ti = ti.replace(forbiddenSymbls[p], '')

        titles.append(ti) # タイトル
        time.sleep(waitTime) 
        foodURLs.append(title.get('href')) # URL
        time.sleep(waitTime)  


    for ing in foodIngredients: # 具材をリストに保管

        n_ings = ing.get_text()

        # 、で要素を分割する
        n_ings = n_ings.split('、')
        for ix in range(len(n_ings)):
            n_ings[ix] = illegal_char_remover(n_ings[ix])
            n_ings[ix] = n_ings[ix].replace('...', '')
        ingredients.append(n_ings)
        time.sleep(waitTime)
        

    
    # 書き込み処理
    for t in range(1, len(titles)+1):

        wb = openpyxl.load_workbook('foods.xlsx')
        ws = wb["Sheet"]

        ws.cell(row=R, column=1).value = titles[t-1] # 料理名を書き込む R行1列

        for idx in range(len(ingredients[t-1])):
            ws.cell(row=R, column=idx+2).value = ingredients[t-1][idx] # 具材を書き込む　R行idx+2(idxは0始まり)列

        ws.cell(row=R, column=idx+3).value = f'https://cookpad.com{foodURLs[t-1]}' # URLを書き込む　R行idx+3(idxは0始まり)列

        wb.save('foods.xlsx')

        # QRコード作成
        FILE_PNG = f'{R-1}.png'
        code = qr.create(f'https://cookpad.com{foodURLs[t-1]}', error='L', version=3, mode='binary')
        code.png(FILE_PNG, scale=5, module_color=[0, 0, 0, 128], background=[255, 255, 255])

        R += 1
    

    dt_now = datetime.datetime.now()
    print(f'{s+1} / {len(url_list)} 完了, {dt_now}')  


print('書き込み完了')